### Write well, Im going to be using this for a long time

#### Data we need:
user input:
- investment amount, trading pair -> amt0, amt1
- start time and end time
- time period that you assume fixed swap price, swap volumes or liquidity positions
- upper and lower price
- pool_fee_rate

data from api:
- cprice of each time period (tick, 1.0001 ** i)
- L_pool at each time period at specific pool_fee_rate (liquidity?, or simply total X tokens + Y tokens in USD)
- Swap volume at each time period at specific pool_fee_rate (volumeUSD?)
- Gas cost to mint at each time period

--------------------------------------------------------------------------------------------------------------

#### Fees
The liquidity amount is calculated from the following numbers that describe a position: 
- amount of token 0 (amt0), amount of token 1 (amt1), 
- price (as x token 1's per token 0) at the upper limit of the position (upper), 
- price at the lower limit of the position (lower) 
- and the current swap price (cprice). 

Then liquidity (L_you?) for a position is calculated as follows:

Case 1: cprice <= lower
- liquidity = amt0 * (sqrt(upper) * sqrt(lower)) / (sqrt(upper) - sqrt(lower))

Case 2: lower < cprice <= upper
- liquidity is the min of the following two calculations:
- amt0 * (sqrt(upper) * sqrt(cprice)) / (sqrt(upper) - sqrt(cprice))
- amt1 / (sqrt(cprice) - sqrt(lower))

Case 3: upper < cprice
- liquidity = amt1 / (sqrt(upper) - sqrt(lower))

Resources
- liquidity can use this code: https://github.com/JNP777/UNI_V3-Liquitidy-amounts-calcs/blob/main/UNI_v3_funcs.py

Fee is calculated by:
- Fee income = (L_you/L_pool) * swap volume under fixed time period * pool_fee_rate/100
- L_you also should be for that specific ticks only, not the whole amount you provided for. Its not linear, its calculated from the 3 cases above
- Does Case1 and Case3's fee be 0 regardless?


reference: https://uniswapv3.flipsidecrypto.com/
- check my numbers with the reference from the website

----------------------------------------------------------------------------------------

#### Impermanent Loss (is this v2 or v3)
- IL (in %) = (2 sqrt(p) / (p+1) ) - 1
- where p = r_t1/r_t2
- and r_t is a price in b at time 1
- Net $ loss = total asset value in dollars at stake time * IL (in%)

reference: https://chainbulletin.com/impermanent-loss-explained-with-examples-math/#:~:text=Impermanent%20loss%20is%20the%20difference,is%20equal%20to%20200%20DAI

--------------------------------------------------------------------------------------------------------------

#### Other cost

Gas_costs_mint = 500000 gwei * gas_price at that time (??? double check actual cost)

### PNL/APR
-PNL = Acumulated Fees_accrued (dolar value at generation) - IL - Gas_costs_mint

-APR = PNL/Initial_capital*(age of the position / year time)

--------------------------------------------------------------------------------------------------------------

## Dependencies

In [1]:
import requests
import json
import pandas as pd
import math
import numpy as np

## Main Functions

In [2]:
# function to use requests.post to make an API call to the subgraph url
def run_query(q):

    # endpoint where you are making the request
    request = requests.post('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
                            '',json={'query': q})
    if request.status_code == 200:
        return request
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))
        
        
# turns requests into dataframe        
# def results_to_df(query_result):
#     json_data_ = json.loads(query_result.text)
#     df_data_ = json_data_['data']['pools']
#     df_ = pd.DataFrame(df_data_)

#     return df_

In [3]:
def get_token_id(symbol):
    
    # default should be first:10, in case there are more than 1 coins with the same symbol
    query_ = """ 
    {{
      tokens(first:1, where:{{symbol: "{}"}}) {{
        id
        symbol
        name
      }}
    }}""".format(symbol)
    
    # run query
    query_result_ = run_query(query_)
    json_data_ = json.loads(query_result_.text)
    
    print(' ')
    print('get_token_id: {}'.format(symbol))
    print(json_data_)
    
    # make sure only return 1 object
    if len(json_data_['data']['tokens']) == 1:
        token_id_ = json_data_['data']['tokens'][0]['id']
        return token_id_
        
    else:
        print(json_data_['data'])
        raise Exception('Returned number of token_ids != 1')

        
def get_pool_id(token0_id, token1_id, feeTier):
    query_ = """
    {{
      pools(first: 10, 
        where:{{token0: "{}",
        token1: "{}",
        feeTier:"{}" }}) 
      {{
        id
        token0{{symbol}}
        token1{{symbol}}
        feeTier
      }}
    }}""".format(token0_id, token1_id, feeTier)
    
    
    # run query
    query_result_ = run_query(query_)
    json_data_ = json.loads(query_result_.text)
    
    print('\n get_pool_id for feeTier: {}'.format(feeTier))
    print(json_data_)
    
    # make sure there is only 1 pool that matches exactly
    if len(json_data_['data']['pools']) == 1:
        pool_id_ = json_data_['data']['pools'][0]['id']
        return pool_id_
    else:
        print(json_data_['data'])
        raise Exception('Returned number of token_ids != 1')

        
    return json_data_

In [4]:
def get_poolDayDatas(pool_id, num_datapoints=1000):
    # input: pool_id
    # num_datapoints (must be multiple of max_request_)
    
    max_request_ = 1000
    quotient_ = math.floor(num_datapoints/max_request_)
#     remainder_ = num_datapoints%max_request_
            
    query_base_ = '''
    {{
      poolDayDatas(first:{},
      skip: {},
        where:{{ pool: "{}" }},
      orderBy:date,
      orderDirection: desc) 
      {{
        date
        tick
        liquidity
        volumeUSD
        pool{{
            token0{{
                symbol
            }}
            token1{{
                symbol
            }}
        }}
      }}
    }}'''
    
    poolDayDatas_array_ = []
    
    # query loop
    for i in range(quotient_):
        q_first_ = max_request_
        q_next_ = i*max_request_
        query_ = query_base_.format(q_first_, q_next_, pool_id)
        query_result_ = run_query(query_)
        json_data_ = json.loads(query_result_.text)
#         print(json_data_)
        poolDayDatas_array_ += json_data_['data']['poolDayDatas']
    
    print(' ')
    print('\n Queried PoolDayDatas, total of {} datapoints'.format(str(len(poolDayDatas_array_))))
#     print('example:')
#     print(poolDayDatas_array_[0])
    
    # array to dataframe
    df_ = pd.json_normalize(poolDayDatas_array_)
    df_.drop_duplicates(subset=['date']) 
    
    return df_

In [5]:
def get_poolHourDatas(pool_id, num_datapoints=3000):
    # input: pool_id
    # num_datapoints (must be multiple of max_request_)
    
    max_request_ = 1000
    quotient_ = math.floor(num_datapoints/max_request_)
#     remainder_ = num_datapoints%max_request_
            
    query_base_ = '''
    {{
      poolHourDatas(first:{},
      skip: {},
        where:{{ pool: "{}" }},
      orderBy:periodStartUnix,
      orderDirection: desc) 
      {{
        periodStartUnix
        pool{{
            token0{{
                symbol
            }}
            token1{{
                symbol
            }}
        }}
        liquidity
        sqrtPrice
        token0Price
        token1Price
        tick
        feeGrowthGlobal0X128
        feeGrowthGlobal1X128
        tvlUSD
        volumeToken0
        volumeToken1
        volumeUSD
        feesUSD
        txCount
        open
        high
        low
        close
      }}
    }}'''
    
    poolDayDatas_array_ = []
    
    # query loop
    for i in range(quotient_):
        q_first_ = max_request_
        q_next_ = i*max_request_
        query_ = query_base_.format(q_first_, q_next_, pool_id)
        query_result_ = run_query(query_)
        json_data_ = json.loads(query_result_.text)
#         print(json_data_)
        try:
            poolDayDatas_array_ += json_data_['data']['poolHourDatas']
        except Exception:
#             print('.. Pass')
            pass
    
    print(' ')
    print('\n Queried poolHourDatas, total of {} datapoints'.format(str(len(poolDayDatas_array_))))
#     print('example:')
#     print(poolDayDatas_array_[0])
    
    # array to dataframe
    df_ = pd.json_normalize(poolDayDatas_array_)
#     df_.drop_duplicates(subset=['periodStartUnix']) # TODO: BUGGGG ??
    
    return df_

In [6]:
def get_swaps(pool_id, time_start='1627369200', time_end='1623772800', num_datapoints=6000):
    # input: pool_id
    # num_datapoints (must be multiple of max_request_)
    
    max_request_ = 1000
    quotient_ = math.floor(num_datapoints/max_request_)
#     remainder_ = num_datapoints%max_request_
           
    query_base_ = '''
    {{
      swaps(first:{}, skip: {},
            where:{{ pool: "{}",
            timestamp_lt: "{}",
            timestamp_gt: "{}"}},
          orderBy:timestamp,
          orderDirection: desc){{
        transaction {{
          blockNumber
          timestamp
          gasUsed
          gasPrice
        }}
        id
        timestamp
        tick
        amount0
        amount1
        amountUSD
        sqrtPriceX96
      }}
    }}'''
    
    swap_arrays_ = []
    
    # query loop
    for i in range(quotient_):
        q_first_ = max_request_
        q_next_ = i*max_request_
        query_ = query_base_.format(q_first_, q_next_, pool_id, time_start, time_end)
        query_result_ = run_query(query_)
        json_data_ = json.loads(query_result_.text)
        
        try:
            swap_arrays_ += json_data_['data']['swaps']
            
        except Exception:
            pass
        
    print('Queried Swaps, total of {} datapoints'.format(str(len(swap_arrays_))))
    df_ = pd.json_normalize(swap_arrays_)
    
    # next time start, if at the edge, then we keep looping and skipping
    if len(swap_arrays_) != 0:
        # last element of timestamp, add 1 so next iterations still includes it
        next_time_start_ = str( int(df_['timestamp'][df_.index[-1]]) + 1 )
    else:
        next_time_start_ = time_start
            
            
    return df_, next_time_start_


# get_swap can only request 6000 datapoints at the time. this is to loop get_swap to get more data
def get_swaps_loop(pool_id, time_start='1627369200', time_end='1623772800',  num_datapoints= 150000):
    
    print('time_start: {}, time_end: {}'.format(time_start, time_end))
    
    max_num_query = 6000
    num_iterations = math.floor(num_datapoints/max_num_query) + 3 # add 3 just in case
    
    next_time_start_ = time_start
    for i in range(num_iterations):
        
        print('next_time_start_: {}'.format(next_time_start_))
        
        df_, next_time_start_ = get_swaps(pool_id, next_time_start_, time_end, num_datapoints=max_num_query)
        
        if i == 0:
            df_all_ = df_
        else:
            df_all_ = df_all_.append(df_)
        
    # drop duplicates
    df_all_ = df_all_.drop_duplicates(subset=['id'])
    
    # reset index
    df_all_ = df_all_.reset_index(drop=True)
    
    return df_all_

In [7]:
def merge_poolHourData_swaps_all(df_poolHourDatas_, df_swaps_all_):
    
    
    return df_merged
    

### Test running

In [8]:
TOKEN0 = 'USDC'
TOKEN1 = 'WETH'
feeTier = '3000' 

In [9]:
# Get token_id > Get pool_id > Get PoolDayDatas > Get swap data > Merge Swap data (VolumeUSD, txCount - for checking)

# Indicate Tokens and FeeTier
token0_id = get_token_id(TOKEN0)
token1_id = get_token_id(TOKEN1)
pool_id = get_pool_id(token0_id, token1_id, feeTier)

 
get_token_id: USDC
{'data': {'tokens': [{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'name': 'USD Coin', 'symbol': 'USDC'}]}}
 
get_token_id: WETH
{'data': {'tokens': [{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'name': 'Wrapped Ether', 'symbol': 'WETH'}]}}

 get_pool_id for feeTier: 3000
{'data': {'pools': [{'feeTier': '3000', 'id': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8', 'token0': {'symbol': 'USDC'}, 'token1': {'symbol': 'WETH'}}]}}


In [10]:
# Get poolHourDatas
df_poolHourDatas = get_poolHourDatas(pool_id, num_datapoints=10000)

# Get Swap Datas within the poolHourDatas timeframe
time_start = df_poolHourDatas['periodStartUnix'][0]
time_end = df_poolHourDatas['periodStartUnix'][df_poolHourDatas.index[-1]]

 

 Queried poolHourDatas, total of 2017 datapoints


In [11]:
# Get Swaps data
df_swaps_all = get_swaps_loop(pool_id, time_start, time_end,  num_datapoints= 150000)

time_start: 1627466400, time_end: 1620169200
next_time_start_: 1627466400
Queried Swaps, total of 6000 datapoints
next_time_start_: 1626771262
Queried Swaps, total of 6000 datapoints
next_time_start_: 1625922309
Queried Swaps, total of 6000 datapoints
next_time_start_: 1625125847
Queried Swaps, total of 6000 datapoints
next_time_start_: 1624613424
Queried Swaps, total of 6000 datapoints
next_time_start_: 1624271840
Queried Swaps, total of 6000 datapoints
next_time_start_: 1623819897
Queried Swaps, total of 6000 datapoints
next_time_start_: 1623265007
Queried Swaps, total of 6000 datapoints
next_time_start_: 1622800990
Queried Swaps, total of 6000 datapoints
next_time_start_: 1622426927
Queried Swaps, total of 6000 datapoints
next_time_start_: 1622217909
Queried Swaps, total of 6000 datapoints
next_time_start_: 1622012255
Queried Swaps, total of 6000 datapoints
next_time_start_: 1621843539
Queried Swaps, total of 6000 datapoints
next_time_start_: 1621679035
Queried Swaps, total of 6000 

In [12]:
# Saving 
# Suffix
SETTINGS = '{}-{}-{}-timestamp-{}-{}.csv'.format(TOKEN0, TOKEN1, feeTier, time_start, time_end)

In [13]:
df_swaps_all.to_csv('../data/df_swaps_all_'+SETTINGS)
df_poolHourDatas.to_csv('../data/df_poolHourDatas_'+SETTINGS)

### Merge Data

In [14]:
# Match timestamp with hour period, and assign to df_swaps_all['periodStartUnix']
def compute_periodStartUnix(row_):
    return row_['timestamp'] - (row_['timestamp'] % 3600)
def compute_periodEndUnix(row_):
    return row_['periodStartUnix'] + 3600

df_swaps_all['timestamp'] = df_swaps_all['timestamp'].astype(int)    
df_swaps_all['periodStartUnix'] = df_swaps_all.apply(lambda row: compute_periodStartUnix(row), axis=1)                       
df_swaps_all['periodEndUnix'] = df_swaps_all.apply(lambda row: compute_periodEndUnix(row), axis=1)                       

df_swaps_all['periodStartUnix'] = df_swaps_all['periodStartUnix'].astype(int)        
df_swaps_all['periodEndUnix'] = df_swaps_all['periodEndUnix'].astype(int)        

In [15]:
# Create swaps_txCount to compare with txCount in poolHourDatas to check integrity
df_swaps_all['swaps_txCount'] = 1

In [16]:
# Groupby->Sum based on periodStartUnix, specify columns to sum at GROUPBY_COLS
GROUPBY_COLS = ['periodStartUnix','amount0', 'amount1', 'amountUSD', 'swaps_txCount']
df_swaps_to_merge = df_swaps_all[GROUPBY_COLS]
df_swaps_to_merge = df_swaps_to_merge.astype({'periodStartUnix': 'int',
                                             'amount0':'float','amount1':'float', 
                                              'amountUSD':'float', 'swaps_txCount':'int'})
df_swaps_to_merge = df_swaps_to_merge.groupby(by=['periodStartUnix']).sum()

In [17]:
# Merge df_swaps_all (groupby) with df_poolHourDatas
df_poolHourDatas['periodStartUnix'] = df_poolHourDatas['periodStartUnix'].astype(int)
df_merged = df_poolHourDatas.merge(df_swaps_to_merge, how='left', on='periodStartUnix')
df_merged['txCount'] = df_merged['txCount'].astype(int)

In [24]:
# Saving data
df_merged.to_csv('../data/df_merged_tmp'+SETTINGS)

In [25]:
watch_list = ['periodStartUnix',  
              'txCount', 'swaps_txCount', # check data integrity
              'amount0', 'amount1', 'amountUSD', # swaps data
              'tick', 'liquidity', 'sqrtPrice', 'tvlUSD', # pool data at that hour
              'pool.token0.symbol', 'pool.token1.symbol', # token data
              'token0Price', 'token1Price'
             ]
df_merged[watch_list]

periodStartUnix  txCount  swaps_txCount       amount0      amount1  \
0          1627466400       14            NaN           NaN          NaN   
1          1627462800       25           15.0  2.122275e+06  -923.169174   
2          1627459200       59           41.0 -1.144813e+06   508.650601   
3          1627455600       61           50.0  6.832045e+06 -2993.768891   
4          1627452000       46           37.0 -7.739540e+06  3409.536166   
...               ...      ...            ...           ...          ...   
2012       1620205200        1            1.0 -3.291690e-01     0.000100   
2013       1620201600        1            1.0 -1.646945e+02     0.050000   
2014       1620180000        3            NaN           NaN          NaN   
2015       1620176400        1            1.0 -3.296080e-01     0.000100   
2016       1620169200        3            NaN           NaN          NaN   

         amountUSD    tick             liquidity  \
0              NaN  198895  18739113134361255191   
1     2.118860e+06  198925  19362935235120456212   
2     6.617698e+06  198971  19028835945343005892   
3     6.820937e+06  198946  19157786910470411251   
4     8.621615e+06  199096  19038318176331007373   
...            ...     ...                   ...   
2012  1.650801e-01  195298      4303369674465501   
2013  8.265004e+01  195298      4303369674465501   
2014           NaN  195285      4533369128072615   
2015  0.000000e+00  195285      4303369674465501   
2016           NaN    None                     0   

                               sqrtPrice                               tvlUSD  \
0     1650546506746906607923223909904123  291954086.5659618997523499505420445   
1     1653010683411328303564197517947849  291443149.9757374234138194528715418   
2     1656804374891287033225664709477372  290854338.3021180896955920486772604   
3     1654757281089015890096203714947244  290357266.3011804428034726800384707   
4     1667243758043164805333369003083165  288636822.8952443253661437502834943   
...                                  ...                                  ...   
2012  1378852426842022799073024911548633  30591.27311273153657547253284189814   
2013  1378850591292581266780357299649652  30466.73734459154650297320284627216   
2014  1377932816571815120446551350158799  30614.68898826013857202593673408213   
2015  1377932816571815120446551350158799                                    0   
2016                                   0                                    0   

     pool.token0.symbol pool.token1.symbol  \
0                  USDC               WETH   
1                  USDC               WETH   
2                  USDC               WETH   
3                  USDC               WETH   
4                  USDC               WETH   
...                 ...                ...   
2012               USDC               WETH   
2013               USDC               WETH   
2014               USDC               WETH   
2015               USDC               WETH   
2016               USDC               WETH   

                              token0Price  \
0     2304.112266171287177872061050692456   
1     2297.247812390796865370198147414455   
2     2286.739544932467783061294104381227   
3     2292.400877784354849685002804199561   
4     2258.192533873747319609614184427765   
...                                   ...   
2012  3301.593431337709250863112750179208   
2013   3301.60222161861579006862518748915   
2014  3306.001762919120095865445515738141   
2015  3306.001762919120095865445515738141   
2016                                    0   

                                  token1Price  
0     0.0004340066300943255467020631083862614  
1     0.0004353034942970640030123199707139924  
2     0.0004373038469623929551020183910980539  
3     0.0004362238776345772991208474685689751  
4     0.0004428320371268700384650205332505981  
...                                       ...  
2012  0.0003028840530479336458756114560460411  
2013  0.000302883

In [27]:
# Eye check:
print(df_poolHourDatas.shape)
print(df_swaps_all.shape)
print(df_merged.shape)

print(df_merged['txCount'].sum())
print(df_merged['swaps_txCount'].sum())

(2017, 20)
(102508, 14)
(2017, 24)
147950
102508.0


##### Query tokens with symbol
{
  tokens(first:10, where:{symbol: "WETH"}) {
    id
    symbol
    name
  }
}

##### Query pools with token0 id,  token1 ids and feeTiers
{
  pools(first:10, 
    where:{token0:"0x6b175474e89094c44da98b954eedeac495271d0f",
    token1: "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",
    feeTier:"3000" }) 
  {
    id
    token0{symbol}
    token1{symbol}
    feeTier
  }
}

##### Query poolDayDatas with pool id, order by date - Needs to be iterative (max 1000 query)
{
  poolDayDatas(first:1000,
  next: 1000,
    where:{pool:"0xa80964c5bbd1a0e95777094420555fead1a26c1e"},
  orderBy:date,
  orderDirection: desc) 
  {
    date
    tick
    liquidity
    volumeUSD
  }
}





##### Query examples on filtering

{
  pools
  (first: 10, 
    where: {liquidity_gt: "1000000", 
      feeTier: "10000"}
    orderBy: liquidity, 
    orderDirection: desc)
  {
    token0{symbol}
    token1{symbol}
    liquidity
  }


(token0) DAI id = 0x6b175474e89094c44da98b954eedeac495271d0f
(token1) WETH id = 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
(feeTier) "3000"

(DAI-WETH 500) Pool id = 0x60594a405d53811d3bc4766596efd80fd545a270
(DAI-WETH 3000) Pool id = 0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8
(DAI-WETH 1000) Pool id = 0xa80964c5bbd1a0e95777094420555fead1a26c1e


